## 網頁的編碼大略上可以分成兩類，ANSI，Unicode
utf-8和big-5都是中文常用的編碼。如果你看一些很古早的網站(大概是20年前)，中文很多都是big-5編碼，現在因為都是用utf-8。所以有時候瀏覽一些中文網站變成亂碼的時候，它很可能是用big-5編碼，那如果你把編碼轉成big-5以後，應該就會正常顯示了。也就是說，如果那個網站還在用big-5的話，有可能十幾二十年都沒有再維護了，資料也可能很舊了，因為我們現在的話中文都是用utf-8編碼。

## ANSI(如BIG5,GBXXXX) :
Big5僅是ANSI的其中一種編碼方式，支援約16000個繁體中文， 可應付大多文書工作，不過若是遇到罕見字，也無法支援，如: 「犇」「鱻」「堃」

Big 5:
1. 每個中文字使用 2 bytes
2. 部分文字用到了控制碼 所以許多程式與軟體都會有許、蓋、功這類中文字的沖碼問題
3. 延伸字集中，有簡體字但是沒有定義日文字

# Unicode(UTF-8, UTF-16)：
UTF-8 :
1. 採用變動大小，中文字使用 3 bytes，英文則維持 1 byte 因此若轉碼後，資料庫整體會變成 1.5 倍大，200MB(Big5) -> 300MB(UTF-8)
2. 支援所有語言文字，還有一些怪怪圖形
3. 簡體與正體中文重複字只定義一次，例如說，簡體的”山”，跟正體的”山” 是同一個 UTF-8 編碼

# 查看目標網頁的編碼

有時候我們爬的網頁可能不是使用UTF-8編碼的, 這時候就需要在程式裡面處理編碼的問題.<br>
那要怎麼看目標網頁的編碼呢? 其實只要透過developer tool觀看網頁的head部分即可, 在head區塊裡, 找到meta標籤, 其中的charset就是該網頁的編碼了, 譬如說以下就是用UTF-8編碼的網頁:

In [ ]:
<meta charset="UTF-8">

# 非UTF-8編碼的處理
若你要爬的網頁, 其編碼不是UTF-8, 就必須在程式裡面處理了, 接下來的範例會示範怎麼做這件事.
至於不同的文件類型, 除了html外, 也許你有一天也會碰上xml, 那這時候就可以選用可處理xml的library去處理其內容.

這邊的範例會去爬兩張非UTF-8編碼的網頁, 其中, 在取得response的時候, 就可以先指定原文件的編碼, 方式如下:

In [ ]:
# 表示爬回來的網頁內容是以BIG-5編碼為基礎的
resp.encoding = 'big5'

最後, 在處理完資料然後要儲存成檔案時, 再轉換成UTF-8編碼:<br>
這樣最後儲存的結果就會是UTF-8編碼了.

In [ ]:
with open('xxx.txt', 'w', encoding='UTF-8') as file:

In [ ]:
<meta http-equiv="content-type" content="text/html;charset=gbk">
<meta charset="utf-8">

In [7]:
import requests
from bs4 import BeautifulSoup


def baidu_encoding():
    resp = requests.get('https://zhidao.baidu.com/question/48795122.html')
    resp.encoding = 'gbk'  #
    soup = BeautifulSoup(resp.text, 'html.parser')
    title = soup.find('span', 'ask-title').text.strip()
    content = soup.find('span', 'con').text.strip().replace('\n', '')
    print('title', title)
    print('content', content)
    try:
        with open(title + '.txt', 'w', encoding='UTF-8') as file:
            file.write(content)
    except Exception as e:
        print(e)


def gold_66_encoding():
    resp = requests.get('http://www.books.com.tw/activity/gold66_day/')
    resp.encoding = 'utf-8' #目前已經從big5改為utf-8
    soup = BeautifulSoup(resp.text, 'html.parser')
    books = list()
    for div in soup.find_all('div', 'mod-04 clearfix'):
        books.append(div.h4.text)
    print('\n'.join(books))
    try:
        with open('66.txt', 'w', encoding='UTF-8') as file:
            file.write('\n'.join(books))
    except Exception as e:
        print(e)


if __name__ == '__main__':
    baidu_encoding()
    print("==============================================")
    gold_66_encoding()

title 网页出现乱码，浏览器字体问题。
content 打开网页全是乱码，点击查看-编码，没有字体可以选择，只有两个选项“从左到右的文档”“从右到左的文档”。点击“Internet选项”，然后点击：“字体”，没有反应。请问怎么办？...
跟任何人都可以用英文聊天：1天1堂英文課，30天融入老外生活圈【虛擬點讀筆版】(附防水書套+超實用必備聊天句300口袋書+1虛擬點讀筆APP+1CD)
一人公司起步的思維與挑戰


### XML文件
這邊用的是ElementTree.XML套件:

In [10]:
import xml.etree.ElementTree as ET


if __name__ == '__main__':
    tree = ET.parse('example.xml')
    root = tree.getroot()
    print(root.attrib)
    total = root.attrib['totalResults']
    movies = list()
    for tag in root.findall('result'):
        print(tag.attrib)
        movies.append(tag.attrib['title'])
    print('-----')
    print('There are', total, 'results in the xml file.')
    print('Top 10 record:')
    print('\n'.join(movies))

FileNotFoundError: [Errno 2] No such file or directory: 'example.xml'